In [ ]:
from pymongo import MongoClient
from lxml import html
import requests
from datetime import date, datetime
import re
import time
from pprint import pprint

In [ ]:
def query(link,params={}):
    """
    Функция содержит общие параметры необходимые для запроса. 
    link – хранит ссылку на сайт, на который адресован запрос.
    params  - необязательный параметр, содержит передаваемые параметры, параметры можно передать и в ссылке link, а params оставить не заполненным.
    Возвращает функция DOM дерево.
    """
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}    
    response = requests.get(link,headers=header,params=params)    
    if params:
        response = requests.get(link,headers=header,params=params)             
    else:
        response = requests.get(link,headers=header)    
    
    dom = html.fromstring(response.text)
    return dom

def pars_date_yandex(tm):
    """
    Функция возвращает временную метку, состоящую из даты запуска функции и переданного в параметре tm времени
    """
    date1=date.today()
    time1=datetime.strptime(tm, '%H:%M')
    datetime1=datetime.combine(date1,time1.time())
    timestamp = datetime1.timestamp()
    return timestamp

def pars(s):
    """
    Функция извлекает время из строки s
    """
    res = re.findall(r'(.*)(?:\s+)(\d{1,2}:\d{1,2})', s)
    return res[0] if res else ()


def retrieved_date_lenta_ru(s):
    """
    Функция извлекает дату из переданной строки s    
    """
    res = re.findall(r'(?:\/)(\d{4})\/(\d{1,2})\/(\d{1,2})', s)
    res = [int(i) for i in res[0]] if res else []    
    return datetime(year=res[0], month=res[1], day=res[2]).timestamp() if res else None

#### Скрапинг новостей с yandex.ru 

In [ ]:
main_link = 'https://yandex.ru/news/'
base_link = 'https://yandex.ru'
res_yandex_ru = []
dom = query(main_link)
items = dom.xpath("//td[@class='stories-set__item']")
for item in items:
    info = {'news_link':"",
        'news_title':"",
        'news_source':"",
        'publication_date':''
    }
    link = item.xpath(".//a[contains(@class,'link_theme_black')]")
    info['news_title'] = link[0].xpath(".//text()")[0]
    info['news_link'] = base_link+link[0].xpath(".//@href")[0]   
    publication_date = pars(item.xpath(".//div[@class='story__date']/text()")[0])
    info['publication_date'] = pars_date_yandex(publication_date[1]) if len(publication_date) == 2 else None
    info['news_source'] = publication_date[0] if len(publication_date) >= 1 else ""
    res_yandex_ru.append(info)    
pprint(len(res_yandex_ru))
pprint(res_yandex_ru)

#### Скрапинг новостей с lenta.ru 

In [ ]:
main_link = 'https://lenta.ru'
base_link = 'https://lenta.ru'
res_lenta_ru = []
dom = query(main_link)

items = dom.xpath("//section[contains(@class,'js-layout')][1]//a[starts-with(@href,'/news/')][not(img)]/parent::*")
for item in items:
    info = {'news_link':"",
        'news_title':"",
        'news_source':"",
        'publication_date':''
    }  
    link = item.xpath(".//a")    
    news_title = link[0].xpath(".//span/text()")
    news_title = news_title[0] if news_title else link[0].xpath("normalize-space(text())")
    info['news_title'] = news_title.replace("\xa0",'') 
    info['news_link'] = base_link+link[0].xpath(".//@href")[0]   
    publication_date = retrieved_date_lenta_ru(info['news_link'])
    info['publication_date'] = retrieved_date_lenta_ru(info['news_link'])
    res_lenta_ru.append(info)

pprint(len(res_lenta_ru))  
pprint(res_lenta_ru)

#### Скрапинг новостей с mail.ru 

In [ ]:
main_link = 'https://news.mail.ru/'
base_link = 'https://news.mail.ru/'
res_mail_ru = []
dom = query(main_link)
items = dom.xpath("//a[contains(@class,'list__text')]|//a[contains(@class,'link_flex')]")
for item in items:
    info = {'news_link':"",
        'news_title':"",
        'news_source':"",
        'publication_date':None
    }
        
    news_link = item.xpath(".//@href")[0]
    info['news_link'] =  news_link if "http" in news_link else base_link + news_link      
    info['news_title'] = item.xpath(".//span[contains(@class,'link__text')]/text()")[0] if item.xpath(".//span") else  item.xpath(".//text()")[0]   
    info['news_title'] = info['news_title'].replace("\xa0",'') 
    
    i = 0
#     Цикл while, в котором реализованна задержка, используется для повторного посещения страницы на которой
#     находятся данные (дата публикации новости и название источника новости), так как сайт не все страницы 
#     отдает с первого раза.

    while i < 3:
        dom1 = query(info['news_link'])  
        item1 = dom1.xpath("//div[contains(@class,'breadcrumbs_article')]")
        item1 = item1[0] if item1 else []
        if item1:
            date_time = item1.xpath(".//span[@datetime]/@datetime")
            news_source = item1.xpath(".//span[contains(@class,'link__text')]/text()")
            info['publication_date'] = datetime.fromisoformat(date_time[0]).timestamp()
            info['news_source'] = news_source[0]
            pprint(info)         
        if info['news_source']:            
            break
        else:            
            i = i + 1
            time.sleep(1)
    res_mail_ru.append(info)    

pprint(len(res_mail_ru))   
pprint(res_mail_ru)    

In [ ]:
client = MongoClient('localhost', 27017)
db = client['news']

In [ ]:
# db.news_yandex.delete_many({})
# db.news_mail.delete_many({})
# db.news_lenta.delete_many({})

In [ ]:
db.news_yandex.insert_many(res_yandex_ru)
db.news_mail.insert_many(res_mail_ru)
db.news_lenta.insert_many(res_lenta_ru)

In [ ]:
for item in db.news_yandex.find({}):
        pprint(item)
        